### 베이즈 정리
조건부 확률

$P(B|A) = P(A|B)P(B)/P(A)$

### 나이브 베이즈 분류

베이지안 필터는 나이브 베이즈 분류 알고리즘을 사용합니다.

베이즈 정리는 A라는 사건이 B에 속하는지를 판단할 때 사용합니다.

스팸분류를 예를들면 A를 입력 텍스트  B를 카테고리 판정 결과라고 할 수 있습니다.

P(A) 는 입력 텍스트가 주어질 확률인데 어떤 카테고리로 판정하든 같은 입력 텍스트가 주어지는 것이므로  P(A) = 1 입니다.

$P(B|A) = P(B) \times P(A|B)$

입력 텍스트 A 를 각 단어의 집합이라고 했을 때

$P(A|B) = P(a_{a1}|B) P(a_{a2}|B) ... P(a_{aN}|B) $

### 베이지안 필터 사용해보기

In [1]:
import math, sys
from konlpy.tag import Twitter

In [3]:
class BayesianFilter:
    """베이지안 필터"""
    def __init__(self):
        self.words = set() # 출현한 단어 기록
        self.word_dict = {} # 카테고리마다의 출현 횟수 기록
        self.category_dict = {} # 카테고리 출현 횟수 기록
        
    def split(self, text):
        results = []
        twitter = Twitter()
        malist = twitter.pos(text, norm=True, stem=True)
        for word in malist:
            if not word[1] in ['Josa', 'Eomi', 'Punctuation']:
                results.append(word[0])
        return results
    
    # 단어와 카테고리의 출현 횟수 세기
    def inc_word(self, word, category):
        if not category in self.word_dict:
            self.word_dict[category] = {}
        if not word in self.word_dict[category]:
            self.word_dict[category][word] = 0
        self.word_dict[category][word] += 1
        self.words.add(word)
        
    def inc_category(self, category):
        if not category in self.category_dict:
            self.category_dict[category] = 0
        self.category_dict[category] += 1
    
    # 주어진 문장의 단어를 특정 카테고리로 분류해서 저장한다.
    def fit(self, text, category):
        word_list = self.split(text)
        for word in word_list:
            self.inc_word(word, category)
        self.inc_category(category)
        
    def score(self, words, category):
        score = math.log(self.category_prob(category))
        for word in words:
            score += math.log(self.word_prob(word, category))
        return score
    
    def predict(self, text):
        best_category = None
        max_score = -sys.maxsize
        words = self.split(text)
        score_list = []
        for category in self.category_dict.keys():
            score = self.score(words, category)
            score_list.append((category, score))
            if score > max_score:
                max_score = score
                best_category = category
        return best_category, score_list
    
    
    # 카테고리 내부의 단어 출현 횟수 구하기
    def get_word_count(self, word, category):
        if word in self.word_dict[category]:
            return self.word_dict[category][word]
        else:
            return 0
        
    # 카테고리 계산
    def category_prob(self, category):
        sum_categories = sum(self.category_dict.values())
        category_v = self.category_dict[category]
        return category_v / sum_categories
    
    # 카테고리 내부의 단어 출현 비율 계산
    def word_prob(self, word, category):
        n = self.get_word_count(word, category) + 1
        d = sum(self.word_dict[category].values()) + len(self.words)
        return n / d

In [5]:
#from bayes import BayesianFilter
bf = BayesianFilter()
# 텍스트 학습
bf.fit("파격 세일 - 오늘까지만 30% 할인", "광고")
bf.fit("쿠폰 선물 & 무료 배송", "광고")
bf.fit("현데계 백화점 세일", "광고")
bf.fit("봄과 함께 찾아온 따뜻한 신제품 소식", "광고")
bf.fit("인기 제품 기간 한정 세일", "광고")
bf.fit("오늘 일정 확인", "중요")
bf.fit("프로젝트 진행 상황 보고","중요")
bf.fit("계약 잘 부탁드립니다","중요")
bf.fit("회의 일정이 등록되었습니다.","중요")
bf.fit("오늘 일정이 없습니다.","중요")
# 예측
pre, scorelist = bf.predict("재고 정리 할인, 무료 배송")
print("결과 =", pre)
print(scorelist)

결과 = 광고
[('광고', -19.00139285840871), ('중요', -20.449365773467083)]
